In [1]:
import numpy as np
import numba

In [2]:
import pint
si = pint.UnitRegistry()

import os
if "ENABLE_DIMENSIONAL_ANALYSIS" not in os.environ:    
    # magic solution by Piotr Bartman, Michael Olesik & SA
    class FakeUnitRegistry:
        @staticmethod
        def __fake(si_unit):
            return (1. * si_unit).to_base_units().magnitude

        def __init__(self, si):
            self.dimensionless = 1.
            for prefix in ["", "hecto", "kilo"]:
                for unit in ["metre", "gram", "joule", "kelvin", "second", "pascal"]:
                    self.__setattr__(prefix+unit, self.__fake(si.__getattr__(prefix+unit)))
                    self.__setattr__(prefix+unit + "s", self.__fake(si.__getattr__(prefix+unit + "s")))

    si = FakeUnitRegistry(si)

In [3]:
g = 9.8 * si.metre / si.second**2
eps = 0.622
L = 2.5e6 * si.joule / si.kilogram
R = 287 * si.joule / si.kilogram / si.kelvin
c_p = 1 * si.kilojoule / si.kilogram / si.kelvin
A = 2.53e8 * si.kilopascals
B = 5.42e3 * si.kelvin

z0 = 0 * si.metres
T0 = 300 * si.kelvin
p0 = 1000 * si.hectopascals

# adiabatic lapse rate (eq. 3.8, R&Y)
gamma = -g / c_p

# saturated vapour pressure (approx!, eq. 2.12, R&Y)
@numba.njit()
def e_s(T):
    return A * np.exp(-B/T)

# w = rho_v / rho_d = ... (eq. 2.18, R&Y)
@numba.njit()
def w_s(T, p):
    e = e_s(T)
    return eps * e / (p - e)

# pseudoadiabatic lapse rate (eq. 3.16, R&Y)
@numba.njit()
def gamma_s(T, p):
    return gamma * (1 + L * w_s(T, p) / R / T) / (1 + L**2 * eps * w_s(T, p) / R / c_p / T**2)

# hydrostatic pressure (approx!, eq. 3.3, R&Y)
@numba.njit()
def p_hydro(T_mean, z, z0, p0):
    return p0 * np.exp(-g / R / T_mean * (z-z0))